<a href="https://colab.research.google.com/github/Liquidator04/Premier-League-Predictor/blob/main/LinearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [19]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

Importing these libraries.

In [20]:
# Load dataset
dftrain = pd.read_csv('https://raw.githubusercontent.com/Liquidator04/Premier-League-Predictor/main/season-1819_csv.csv')
dfeval = pd.read_csv('https://raw.githubusercontent.com/Liquidator04/Premier-League-Predictor/main/17_18_test_csv.csv')
y_train = dftrain.pop('FTHG')
y_eval = dfeval.pop('FTHG')

dftrain and dfeval are the dataframe a pandas datastructures.

In [21]:
dftrain.head()

,Div,Date,HomeTeam,AwayTeam,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
0,E0,10/08/2018,Man United,Leicester,1,H,1,0,H,A Marriner,...,1.79,17,-0.75,1.75,1.70,2.29,2.21,1.55,4.07,7.69
1,E0,11/08/2018,Bournemouth,Cardiff,0,H,1,0,H,K Friend,...,1.83,20,-0.75,2.20,2.13,1.80,1.75,1.88,3.61,4.70
2,E0,11/08/2018,Fulham,Crystal Palace,2,A,0,1,A,M Dean,...,1.87,22,-0.25,2.18,2.11,1.81,1.77,2.62,3.38,2.90
3,E0,11/08/2018,Huddersfield,Chelsea,3,A,0,2,A,C Kavanagh,...,1.84,23,1.00,1.84,1.80,2.13,2.06,7.24,3.95,1.58
4,E0,11/08/2018,Newcastle,Tottenham,2,A,1,2,A,M Atkinson,...,1.81,20,0.25,2.20,2.12,1.80,1.76,4.74,3.53,1.89


In [22]:
CATEGORICAL_COLUMNS = ['HomeTeam']
NUMERICAL_COLUMNS = ['HS','HST','HC','HR','HY']
feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))
for feature_name in NUMERICAL_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name,dtype=tf.float32))
print(feature_columns)

[VocabularyListCategoricalColumn(key='HomeTeam', vocabulary_list=('Man United', 'Bournemouth', 'Fulham', 'Huddersfield', 'Newcastle', 'Watford', 'Wolves', 'Arsenal', 'Liverpool', 'Southampton', 'Cardiff', 'Chelsea', 'Everton', 'Leicester', 'Tottenham', 'West Ham', 'Brighton', 'Burnley', 'Man City', 'Crystal Palace'), dtype=tf.string, default_value=-1, num_oov_buckets=0), NumericColumn(key='HS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='HST', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='HC', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='HR', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='HY', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [23]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():  # inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle(1000)  # randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
    return ds  # return a batch of the dataset
  return input_function  # return a function object for use

train_input_fn = make_input_fn(dftrain, y_train)  # here we will call the input_function that was returned to us to get a dataset object we can feed to the model
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

In [24]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns,n_classes=len(y_train.unique()))

In [25]:
linear_est.train(train_input_fn)  # train
result = linear_est.evaluate(eval_input_fn)  # get model metrics/stats by testing on tetsing data

clear_output()  # clears consoke output
print(result['accuracy'])  # the result variable is simply a dict of stats about our model

0.4105263


In [33]:
result = list(linear_est.predict(eval_input_fn))
print(result[18]['probabilities'])

[2.5395799e-01 5.1109159e-01 1.6339083e-01 3.9645419e-02 2.9856592e-02
 1.8903411e-03 1.6722955e-04]
